In [213]:
import numpy as np

In [104]:
# Policy parameterization: Softmax
# State value parameterization: Linear
# State feature vector representation: Fourier (as in HW2)

In [254]:
M = 3 # dimensionality of the fourier transform [2, 10]
# mean_policy_vector = np.ones(2*M+1)*0.05 #some non-zero vector

policy_params = np.ones((2*M+1,2))*0.01
value_params = np.ones(2*M+1)*0.01

softmax_sigma = 0.1
# policy_params = np.zeros((np.power(M+1,2),2))
# value_params = np.zeros(np.power(M+1,2))

policy_step_size = 1e-7 #2e-9
valueFunction_step_size = 1e-7
gamma = 1

In [215]:
def phi_s_cosine(given_x, given_v, cosineFlag):
    normal_x = normalized_x(given_x, cosineFlag)
    normal_v = normalized_v(given_v, cosineFlag)
    feature_phi = np.zeros(2*M+1)
#     feature_phi = np.zeros(np.power(M+1,2))
    feature_phi[0] = 1
    if cosineFlag:
        for i in range(1, M+1):
            feature_phi[i] = np.cos(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.cos(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
    
    else:
        for i in range(1, M+1):
            feature_phi[i] = np.sin(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.sin(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))

    return feature_phi

In [216]:
def normalized_x(given_x, cosineFlag):
    if cosineFlag:
        return (given_x+1.2)/(0.5+1.2)
    else:
        return ((given_x+1.2)/(0.5+1.2))*2 - 1
def normalized_v(given_v, cosineFlag):
    if cosineFlag:
        return (given_v+0.7)/(0.7+0.7)
    else:
        return ((given_v+0.7)/(0.7+0.7))*2 - 1

In [248]:
def runEpisode(policy_params, gamma):
    
    episode_trace = []
    
    total_reward = 0
    x_0 = np.random.uniform(-0.6,-0.4,1)[0]
    begin_state = (x_0,0)
    
    S_t = begin_state
    S_t_plus_1 = begin_state
    for t in range(0, 1000):
        
        state_feature_vector = phi_s_cosine(S_t[0], S_t[1], True)
        state_feature_policy_param_product = np.dot(policy_params.T, state_feature_vector)
        exp_terms = np.exp(softmax_sigma * state_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
        A_t = np.random.choice([-1,1], p=list(pi_values))
        
        max_action_idx = np.argmax(pi_values)
        if max_action_idx==1:
            A_t = 1
        else:
            A_t = -1
        
#         threshold = np.dot(state_feature_vector, given_theta)
        
#         print(threshold)
        
#         if threshold<=0:
#             A_t = -1
#         else:
#             A_t = 1
        
        v_t_plus_1 = S_t[1] + 0.001*A_t - 0.0025*np.cos(3*S_t[0])
        x_t_plus_1 = S_t[0] + v_t_plus_1
        
        
        
        if x_t_plus_1 < -1.2:
            x_t_plus_1 = -1.2
            v_t_plus_1 = 0
        elif x_t_plus_1 > 0.5:
            x_t_plus_1 = 0.5
            v_t_plus_1 = 0
        
        if v_t_plus_1 < -0.7:
            v_t_plus_1 = -0.7
        elif v_t_plus_1 > 0.7:
            v_t_plus_1 = 0.7
        
        S_t_plus_1 = (x_t_plus_1, v_t_plus_1)
        
        if x_t_plus_1 == 0.5:
            print("Terminated")
            episode_trace.append((S_t, A_t, 0))
            break
        else:
            episode_trace.append((S_t, A_t, -1))
            total_reward += -1
#         print("After ",total_reward)
        

        S_t = S_t_plus_1 #Incrementing time step
    return episode_trace

In [109]:
curr_iter = 0
while(curr_iter<=1000):
    curr_iter += 1
    
    episode_trace = runEpisode(policy_params, gamma)
    print("\n EPISODE LENGTH: ",len(episode_trace), "CURR ITER: ", curr_iter,"\n")
    for a_step in range(len(episode_trace)):
        G = 0
        for idx in range(a_step, len(episode_trace)):
            G += episode_trace[idx][2]
        
        state_t = episode_trace[a_step][0]
        state_t_feature_vector = phi_s_cosine(state_t[0], state_t[1], True)
        v_hat_s = np.dot(state_t_feature_vector, value_params)
        delta = G - v_hat_s
        
        value_params += valueFunction_step_size*delta*state_t_feature_vector
        
#         print("Policy params: ",policy_params)
        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
#         print("State feature product: ",state_t_feature_policy_param_product)
        
        exp_terms = np.exp(softmax_sigma * state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
        print(policy_step_size*delta, ". Pi values: ",pi_values)
        
        if episode_trace[a_step][1] == -1:
            policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
        else:
            policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    


 EPISODE LENGTH:  1000 CURR ITER:  1 

-1.9998688228202135 . Pi values:  [0.5 0.5]
-1.9965514666934001 . Pi values:  [0.99862588 0.00137412]
-1.993235039398439 . Pi values:  [0.50333604 0.49666396]
-1.9899171311761983 . Pi values:  [0.00141275 0.99858725]
-1.9866005242259552 . Pi values:  [0.50223705 0.49776295]
-1.9832791305910524 . Pi values:  [0.00140759 0.99859241]
-1.9799511303774897 . Pi values:  [1.93037167e-06 9.99998070e-01]
-1.9766144663436285 . Pi values:  [2.55642628e-09 9.99999997e-01]
-1.9732766316404415 . Pi values:  [1.84978459e-06 9.99998150e-01]
-1.9699292763093235 . Pi values:  [2.37580053e-09 9.99999998e-01]
-1.9665820970194523 . Pi values:  [1.78709530e-06 9.99998213e-01]
-1.9632260519608151 . Pi values:  [2.22916441e-09 9.99999998e-01]
-1.9598625339960607 . Pi values:  [2.67715447e-12 1.00000000e+00]
-1.9565012272146496 . Pi values:  [2.12662764e-09 9.99999998e-01]
-1.9531342301476726 . Pi values:  [2.49129843e-12 1.00000000e+00]
-1.949764923643925 . Pi values:  

-1.1268171374625664 . Pi values:  [1.00000000e+00 5.69438732e-47]
-1.1220405861665808 . Pi values:  [1.0000000e+00 5.2319418e-49]
-1.1170552649916652 . Pi values:  [1.00000000e+00 5.34599762e-48]
-1.1119428001940077 . Pi values:  [1.00000000e+00 5.13800683e-47]
-1.1071596968270654 . Pi values:  [1.00000000e+00 4.19208228e-49]
-1.1027226988564807 . Pi values:  [1.00000000e+00 3.90042106e-51]
-1.0986034685493795 . Pi values:  [1.00000000e+00 4.13030917e-53]
-1.0945498367698687 . Pi values:  [1.00000000e+00 6.74291443e-52]
-1.0907874142395737 . Pi values:  [1.00000000e+00 8.01453277e-54]
-1.0871774096831255 . Pi values:  [1.00000000e+00 1.64680498e-52]
-1.0838381415296805 . Pi values:  [1.000000e+00 2.293394e-54]
-1.080711624277698 . Pi values:  [1.00000000e+00 6.04634706e-53]
-1.0778062198723029 . Pi values:  [1.00000000e+00 1.00875427e-54]
-1.0750782870746023 . Pi values:  [1.00000000e+00 1.87532318e-56]
-1.0724543931046129 . Pi values:  [1.00000000e+00 3.79708805e-58]
-1.06998339210965

-0.4930540655110185 . Pi values:  [1.00000000e+00 6.19233547e-81]
-0.4906739482078325 . Pi values:  [1.00000000e+00 3.64460947e-80]
-0.48831766543230687 . Pi values:  [1.00000000e+00 7.45698846e-81]
-0.4859774336526336 . Pi values:  [1.00000000e+00 4.47001315e-80]
-0.48365084831401745 . Pi values:  [1.00000000e+00 2.58979839e-79]
-0.4813786140002779 . Pi values:  [1.00000000e+00 5.60367397e-80]
-0.4790988725683124 . Pi values:  [1.00000000e+00 3.30294257e-79]
-0.4769072943828308 . Pi values:  [1.0000000e+00 7.6737149e-80]
-0.47469681839687566 . Pi values:  [1.00000000e+00 4.62948101e-79]
-0.4724620249055001 . Pi values:  [1.00000000e+00 2.64411002e-78]
-0.470335620830175 . Pi values:  [1.00000000e+00 6.36881678e-79]
-0.4681569588195344 . Pi values:  [1.00000000e+00 3.68773594e-78]
-0.4659219412735832 . Pi values:  [1.00000000e+00 1.99196469e-77]
-0.46380537503090613 . Pi values:  [1.00000000e+00 4.82156337e-78]
-0.4618257962933523 . Pi values:  [1.00000000e+00 1.30498507e-78]
-0.459764

0.19510329644810667 . Pi values:  [1.00000000e+00 5.95197962e-79]
0.1971966699899315 . Pi values:  [1.00000000e+00 8.99579227e-79]
0.19917174481883473 . Pi values:  [1.00000000e+00 1.51313287e-78]
0.20107947017662525 . Pi values:  [1.00000000e+00 2.77174667e-78]
0.20295371489497974 . Pi values:  [1.00000000e+00 5.44894578e-78]
0.20480983109831788 . Pi values:  [1.00000000e+00 1.14147512e-77]
0.20667090884314582 . Pi values:  [1.00000000e+00 5.40689204e-78]
0.20851784381127061 . Pi values:  [1.00000000e+00 1.17510267e-77]
0.21037304395034495 . Pi values:  [1.00000000e+00 5.61986949e-78]
0.21223214862997794 . Pi values:  [1.00000000e+00 2.58807803e-78]
0.2140890669074347 . Pi values:  [1.00000000e+00 1.15028533e-78]
0.21593142965239695 . Pi values:  [1.00000000e+00 4.95968222e-79]
0.21777749967100646 . Pi values:  [1.00000000e+00 1.04356185e-78]
0.2196032626507066 . Pi values:  [1.00000000e+00 4.57263186e-79]
0.2214477793727665 . Pi values:  [1.00000000e+00 9.99073425e-79]
0.223306138329

-1.3558494244083372 . Pi values:  [3.64664657e-72 1.00000000e+00]
-1.3545025574596898 . Pi values:  [1.35865403e-73 1.00000000e+00]
-1.3530194355933254 . Pi values:  [4.35795272e-75 1.00000000e+00]
-1.3513149078678854 . Pi values:  [1.16418185e-76 1.00000000e+00]
-1.3493201632653713 . Pi values:  [2.51516434e-78 1.00000000e+00]
-1.3469843833401651 . Pi values:  [4.28760047e-80 1.00000000e+00]
-1.3442758859552826 . Pi values:  [5.65805205e-82 1.00000000e+00]
-1.3411828255313543 . Pi values:  [5.70609368e-84 1.00000000e+00]
-1.3377134968074655 . Pi values:  [4.37144945e-86 1.00000000e+00]
-1.3338962607156821 . Pi values:  [2.54766448e-88 1.00000000e+00]
-1.3297790784389465 . Pi values:  [1.14008688e-90 1.00000000e+00]
-1.3254286092248435 . Pi values:  [3.98725132e-93 1.00000000e+00]
-1.3209288050581953 . Pi values:  [1.11879893e-95 1.00000000e+00]
-1.3163789261538443 . Pi values:  [2.60839521e-98 1.00000000e+00]
-1.311890909462323 . Pi values:  [5.2781976e-101 1.0000000e+000]
-1.30758605

...

0.3602144245611903 . Pi values:  [3.41035668e-33 1.00000000e+00]
0.36143771741509023 . Pi values:  [5.77658289e-24 1.00000000e+00]
0.36102078381405817 . Pi values:  [1.73706689e-13 1.00000000e+00]
0.35863973849081593 . Pi values:  [0.07957888 0.92042112]
0.3551052935654757 . Pi values:  [1.00000000e+00 6.40310608e-12]
0.35064602560073443 . Pi values:  [1.00000000e+00 1.84814026e-23]
0.34570121563728706 . Pi values:  [1.00000000e+00 4.23859122e-34]
0.34068595333171925 . Pi values:  [1.00000000e+00 1.09889953e-43]
0.3359780672638235 . Pi values:  [1.00000000e+00 4.27638543e-52]
0.33190861558788176 . Pi values:  [1.0000000e+00 3.1084497e-59]
0.3287555852931496 . Pi values:  [1.00000000e+00 4.95750835e-65]
0.32674019765152323 . Pi values:  [1.00000000e+00 1.94016198e-69]
0.32602510018817227 . Pi values:  [1.00000000e+00 2.00151165e-72]
0.3267137316421565 . Pi values:  [1.00000000e+00 5.65823461e-74]
0.32885025452377864 . Pi values:  [1.00000000e+00 4.43374843e-74]
0.33241963343409475 . Pi 

0.8788429462701663 . Pi values:  [1.00000000e+00 3.05532877e-29]
0.8812235936804678 . Pi values:  [1.00000000e+00 1.09884699e-28]
0.8842352031394789 . Pi values:  [1.00000000e+00 7.44407002e-27]
0.8877873145869263 . Pi values:  [1.00000000e+00 8.62312516e-24]

 EPISODE LENGTH:  1000 CURR ITER:  10 

-1.1490860334766162 . Pi values:  [1.0000000e+00 9.6941207e-84]
-1.1450087422049566 . Pi values:  [1.00000000e+00 1.78098999e-80]
-1.1396362106605715 . Pi values:  [1.00000000e+00 1.58275321e-75]
-1.1330733351712834 . Pi values:  [1.00000000e+00 6.47353783e-69]
-1.1254783662939336 . Pi values:  [1.00000000e+00 1.11612572e-60]
-1.1170627730674527 . Pi values:  [1.00000000e+00 7.07527601e-51]
-1.1080900250887835 . Pi values:  [1.00000000e+00 1.35268417e-39]
-1.0988725402005162 . Pi values:  [1.00000000e+00 5.94387477e-27]
-1.0897660432388647 . Pi values:  [1.00000000e+00 4.20752986e-13]
-1.0811607051719938 . Pi values:  [0.0032413 0.9967587]
-1.0734686941689011 . Pi values:  [7.35083579e-19 1

-0.6819518863342223 . Pi values:  [1.00000000e+00 3.13042229e-23]
-0.6850296024627283 . Pi values:  [1.00000000e+00 1.71147637e-36]
-0.6888414830590219 . Pi values:  [1.00000000e+00 5.40398794e-47]
-0.6931560183623555 . Pi values:  [1.00000000e+00 1.42985537e-56]
-0.6975583422071377 . Pi values:  [1.00000000e+00 4.52306311e-65]
-0.7016700824878669 . Pi values:  [1.00000000e+00 2.27855266e-72]
-0.7051590444792891 . Pi values:  [1.00000000e+00 2.28222522e-78]
-0.7077455457051666 . Pi values:  [1.00000000e+00 5.35701081e-83]
-0.7092060308237101 . Pi values:  [1.00000000e+00 3.30888199e-86]
-0.7093747289691241 . Pi values:  [1.00000000e+00 5.80703217e-88]
-0.7081441177214806 . Pi values:  [1.00000000e+00 3.03116467e-88]
-0.7054648505174439 . Pi values:  [1.00000000e+00 4.80443796e-87]
-0.7013456165666556 . Pi values:  [1.000000e+00 2.309062e-84]
-0.6958531635523593 . Pi values:  [1.00000000e+00 3.28495027e-80]
-0.6891124513883768 . Pi values:  [1.00000000e+00 1.31495872e-74]
-0.68130664651

-0.12583263949091805 . Pi values:  [1.00000000e+00 1.62281796e-88]
-0.11926209091912608 . Pi values:  [1.00000000e+00 2.60396934e-84]
-0.11141400820417038 . Pi values:  [1.0000000e+00 9.4342827e-79]
-0.10253672997780723 . Pi values:  [1.00000000e+00 6.97205106e-72]
-0.09294127119460541 . Pi values:  [1.00000000e+00 9.07497362e-64]
-0.0829978249002686 . Pi values:  [1.00000000e+00 1.70012966e-54]
-0.07312920544838312 . Pi values:  [1.00000000e+00 3.51590906e-44]
-0.06380045590044006 . Pi values:  [1.00000000e+00 5.74811786e-33]
-0.05550415238963024 . Pi values:  [1.00000000e+00 4.96663209e-21]
-0.04901589418239587 . Pi values:  [1.00000000e+00 3.03336371e-10]
-0.044293051971621485 . Pi values:  [0.35664567 0.64335433]
-0.04105132159634104 . Pi values:  [1.06308647e-09 9.99999999e-01]
-0.03897831273416318 . Pi values:  [2.43701525e-17 1.00000000e+00]
-0.037751155200678566 . Pi values:  [6.68039861e-24 1.00000000e+00]
-0.03705327927933854 . Pi values:  [2.11651447e-29 1.00000000e+00]
-0.0

KeyboardInterrupt: 

In [ ]:
#Without baseline

curr_iter = 0
while(curr_iter<=5000):
    curr_iter += 1
    
    episode_trace = runEpisode(policy_params, gamma)
    print("\n EPISODE LENGTH: ",len(episode_trace), "CURR ITER: ", curr_iter)
    policy_params_temp = policy_params.copy()
    for a_step in range(len(episode_trace)):
        G = 0
        for idx in range(a_step, len(episode_trace)):
            G += episode_trace[idx][2]
        
        state_t = episode_trace[a_step][0]
        state_t_feature_vector = phi_s_cosine(state_t[0], state_t[1], True)

        v_hat_s = np.dot(state_t_feature_vector, value_params)
        delta = G - v_hat_s
        
        value_params += valueFunction_step_size*delta*state_t_feature_vector
        

        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
        exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         print("Pi values: ",pi_values)
#         print("G: ",G,". Delta: ",delta)
        if episode_trace[a_step][1] == -1:
            policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
        else:
            policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    max_diff = np.max(np.abs(policy_params_temp - policy_params))
    print(policy_step_size," Max diff: ",max_diff)
#     if max_diff<policy_step_size*1e2 or len(episode_trace)<110:# 0.001 works with 1e-6 policy_step
#         break